In [0]:
from pyspark.sql.functions import col
from pyspark.sql.functions import lower
from pyspark.sql.functions import to_timestamp, lit,sum,coalesce
from pyspark.sql.functions import col, when, lit, current_timestamp, to_date, year, month, upper, trim,count,max,countDistinct,min,desc

df= spark.table("workspace.default.customers")

display(df)
#************select & filters
#df_select = df.select("CustomerId", "CustomerName", "Country")
#display(df_select)

##usa = df.filter(col("Country").isin("USA","India"))
#display(usa)

#name_has_a = df.filter(lower(col("CustomerName")).contains("a"  ))
#display(name_has_a)

#greaterthan3 = df.filter(col("CustomerId") > 3)
#display(greaterthan3)

#afterdate= df.filter(col("LastUpdatedDate") > to_timestamp(lit("2019-01-01")))
#display(afterdate)

#usa_ind = df.filter((col("Country").isin("USA","Canada")) & (col("CustomerId")>2)).orderBy(col("LastUpdatedDate").#desc())
#display(usa_ind)
#df.printSchema()
#print(df.count())

#*********** add new column , when otherwise
df_newcolumn = df.withColumn("IsUSA", when(col("Country") == ("USA"), lit(1)).otherwise(lit(0)))
#display(df_newcolumn)

df2 = (df_newcolumn.withColumn("CustomerCleanName",trim(col("CustomerName")))
       .withColumn("CountryUpper",upper(trim(col("Country")))))
#display(df2)

df3 = (df2
  .withColumn("UpdatedDate", to_date(col("LastUpdatedDate")))
  .withColumn("UpdatedYear", year(col("LastUpdatedDate")))
  .withColumn("UpdatedMonth", month(col("LastUpdatedDate")))
)
#display(df3)

df4 = df3.withColumn("IngestedAt", current_timestamp())
#display(df4)

df5 = df.withColumn("CustomerTier", when(col("Country") == "USA","tier1").
                                    when(col("Country") == "Canada","tier2").otherwise("tier3"))
#display(df5)

#************groupby aggregation
by_country = df.groupBy("Country").agg(count(col("Country")).alias("numberofcustomer")).orderBy(col("numberofcustomer").desc())
#display(by_country)

latest_per_country = (df
  .groupBy("Country")
  .agg(max("LastUpdatedDate").alias("LatestUpdate"))
  .orderBy(col("LatestUpdate").desc())
)
#display(latest_per_country)

multi = (df
  .groupBy("Country")
  .agg(
      count("*").alias("Rows"),
      countDistinct("CustomerId").alias("DistinctCustomers"),
      min("LastUpdatedDate").alias("OldestUpdate"),
      max("LastUpdatedDate").alias("NewestUpdate")
  )
  .orderBy(col("Rows").desc())
)
#display(multi)

by_month= (df
           .groupby(month(col("LastUpdatedDate")))
           .agg(count("*").alias("Rows"))
           .orderBy(month(col("LastUpdatedDate")))
)
#display(by_month)

task2= (df
.groupby(col("Country"))
.agg(count(col("CustomerId")).alias("CustomerCount"))
.orderBy(col("CustomerCount").desc())
)
#display(task2)

#************Joins
orders_data = [
    (101, 1, 120.0, "2025-12-01"),
    (102, 1,  75.0, "2025-12-02"),
    (103, 2,  50.0, "2025-12-03"),
    (104, 4, 200.0, "2025-12-05"),
    (105, 99, 60.0, "2025-12-06")  # customerId 99 doesn't exist -> join test
]
orders = spark.createDataFrame(orders_data, ["OrderId", "CustomerId", "Amount", "OrderDate"]) \
             .withColumn("OrderDate", to_date("OrderDate"))
#display(orders)


inner_join = df.join(orders, on="CustomerId",how="inner")
#display(inner_join)

left_join = df.join(orders, on="CustomerId",how="left")
#display(left_join)  

right_join = df.join(orders, on="CustomerId",how="right")
#display(right_join)

cust_spend = (df
  .join(orders, on="CustomerId", how="left")
  .groupBy("CustomerId", "CustomerName", "Country")
  .agg(
      count("OrderId").alias("OrderCount"),
      coalesce(sum("Amount"), lit(0.0)).alias("TotalAmount")
  )
  .orderBy(col("TotalAmount").desc())
)

#display(cust_spend)

country_order_revenue= (df
                        .join(orders,on="CustomerId",how="inner")
                        .groupBy("Country")
                        .agg(count("*").alias("TotalOrders")
                            ,sum("Amount").alias("TotalRevenue"))
                        .orderBy(col("TotalRevenue").desc())
)
display(country_order_revenue)


#**************to write the output in another file
#spark.sql("SHOW VOLUMES IN workspace.default").show()

#out = "/Volumes/workspace/default/mainvol/deepika/week1/customers_parquet"
#df.write.mode("overwrite").parquet(out)

#df_back = spark.read.parquet(out)
#display(df_back)
#print(df_back.count())


